In [158]:
import os
import json
import pandas as pd
import traceback
from langchain.llms import OpenAI

In [159]:
from langchain.chat_models import ChatOpenAI

In [160]:
from dotenv import load_dotenv

load_dotenv()

True

In [161]:
KEY=os.getenv("OPENAI_API_KEY")

In [162]:
llm = ChatOpenAI(openai_api_key=KEY, model="gpt-5", temperature=1)

In [163]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import pandas as pd
import traceback
import pypdf


In [164]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [165]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [166]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "response_json"],
    template=TEMPLATE)

In [167]:
quiz_chain= LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [168]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for analysis\
If the quiz is not as per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student\
QUIZ_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [169]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE)

In [170]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [171]:
generate_evaluation_chain=SequentialChain(chains=[quiz_chain, review_chain],
                                          input_variables=["text", "number", "subject", "tone", "response_json"],
                                          output_variables=["quiz", "review"],
                                          verbose=True)

In [172]:
file_path=r'C:\Users\HP\mcqgen\experiment\data.txt'

In [173]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [174]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[4][5]

From a theoretical viewpoint, probably approximately c

In [175]:
#serialize the pythin dict to json string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [176]:
NUMBER= 5
SUBJECT="machine learning"
TONE="simple and easy to understand"

In [178]:
# How to setup Token Usage Tracking in LangChain
# https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking
with get_openai_callback() as cb:
    response = generate_evaluation_chain({
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory 

In [179]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")


Total Tokens: 5509
Prompt Tokens: 868
Completion Tokens: 4641
Total Cost (USD): $0.0


In [181]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[4][5]\n\nFrom a theoretical viewpoint, probably

In [182]:
quiz=response.get("quiz")

In [184]:
quiz= json.loads(quiz)

In [185]:
quiz_table_data = []

for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [186]:
quiz_table_data

[{'MCQ': 'What is machine learning (ML) mainly concerned with?',
  'Choices': 'a: Developing statistical algorithms that learn from data and generalise to unseen data | b: Writing explicit instructions for every possible task | c: Creating only rule-based expert systems | d: Simulating human emotions and consciousness',
  'Correct': 'Developing statistical algorithms that learn from data and generalise to unseen data'},
 {'MCQ': 'Advances in which subfield have enabled neural networks to surpass many previous ML approaches in performance?',
  'Choices': 'a: Deep learning | b: Linear regression | c: Symbolic AI | d: Database indexing',
  'Correct': 'Deep learning'},
 {'MCQ': 'What is the application of ML to business problems called?',
  'Choices': 'a: Predictive analytics | b: Exploratory data analysis | c: Operations research | d: Data warehousing',
  'Correct': 'Predictive analytics'},
 {'MCQ': 'Which areas form the foundations of machine learning?',
  'Choices': 'a: Statistics and m

In [188]:
quiz= pd.DataFrame(quiz_table_data)

In [189]:
quiz.to_csv('mcq_output.csv', index=False)